In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from IPython.display import Image


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint


from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn import model_selection,preprocessing
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.inspection import PartialDependenceDisplay
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay


%matplotlib inline

## Step 1 - Geant4 simulation (20%)

Creating a detector that consists of:

A homogenous EM calorimeter (ECAL) made from lead-glass (10cm in length)

A sampling hadronic calorimeter (HCAL): 5 lead (3cm) and 5 liquid argon (30cm) layers alternating 

## Step 2 - Data reconstruction (20%)

To detect the particle energies the ECAL and HCAL are made into sensitive detectors

The simulation is ran separately for a particle gun of electrons, photons, protons and neutrons. The particle gun energy is incremented by 5 MeV for every particle being fired. This allows to obtain a range of particle energies. 5000 particles are fired for each particle type with the energy starting at 200 MeV and finishing at 25 GeV. This produces 20000 events in total.

The simulation precision was slightly reduced to speed up the computation times.

In [ ]:
col_names = ["TrueEnergy", "ECAL","HLayer1", "HLayer2", "HLayer3", "HLayer4", "HLayer5"]

In [ ]:
layers = col_names[1:]

### Load the particles

In [ ]:
electrons = pd.read_csv("Electrons2.csv",comment="#",
                                header=None,names=col_names,dtype="float")

In [ ]:
photons = pd.read_csv("Photons2.csv",comment="#",
                                header=None,names=col_names,dtype="float")

In [ ]:
protons = pd.read_csv("Protons2.csv",comment="#",
                                header=None,names=col_names,dtype="float")

In [ ]:
neutrons = pd.read_csv("Neutrons2.csv",comment="#",
                                header=None,names=col_names,dtype="float")

### Give labels

Particle IDs

electron = 0, photon = 1, proton = 2, neutron = 3

In [ ]:
electrons.insert(loc=0,column="ID",value=np.zeros(electrons.shape[0]))
photons.insert(loc=0,column="ID",value=np.ones(protons.shape[0]))
protons.insert(loc=0,column="ID",value=2*np.ones(protons.shape[0]))
neutrons.insert(loc=0,column="ID",value=3*np.ones(protons.shape[0]))

### Combine datasets

In [ ]:
particles = pd.concat([electrons,photons,protons,neutrons],ignore_index=True)
particles.reset_index(drop=True, inplace=True)

In [ ]:
particles

### Create training and testing datasets

In [ ]:
# Split into features and target sets
features = particles[particles.columns[2:]]
target = particles.ID

In [ ]:
# Standarise data and split into training and test set
sc = preprocessing.StandardScaler()
features = sc.fit_transform(features)
# set random seed
seed = 1
# train - test split of dataset
x_train, x_test, y_train, y_test, train_energy, test_energy = \
model_selection.train_test_split(features, target, particles.TrueEnergy, test_size=0.3,random_state = seed)
print (x_train.shape, x_test.shape, y_train.shape,y_test.shape)

In [ ]:
def my_model(num_inputs, num_nodes, extra_depth,drop_out):
    # create model
    model = Sequential()
    # Input shape equal to number of features (6 detector readouts)
    model.add(Dense(num_nodes, input_dim=num_inputs, kernel_initializer="normal", activation="relu"))
    # Add dropout to prevent overtraining
    model.add(Dropout(drop_out))
    for i in range (extra_depth):
        # Adding dense layers
        model.add(Dense(num_nodes, kernel_initializer="normal", activation="relu"))
        # Following each dense layer by dropout layer
        model.add(Dropout(drop_out))
    model.add(Dense(4, activation="softmax",kernel_initializer="normal"))
    # Compile model to use multi-class loss
    model.compile(loss="sparse_categorical_crossentropy",optimizer="adam", metrics =["accuracy"])
    return model

In [ ]:
# Create the model with chosen hyper parameters
num_nodes = 50
extra_depth = 2
drop_out=0.2
model = my_model(x_train.shape[1],num_nodes,extra_depth,drop_out)

In [ ]:
model.summary()

In [ ]:
# Set training hyperparameters
batchSize = 50
N_epochs = 50

In [ ]:
# train the model and save the training history
history = model.fit(x_train,y_train,batch_size=batchSize,epochs=N_epochs,verbose=1,
                    validation_data=(x_test,y_test))

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend(["loss","val_loss"])
plt.show()


plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model Validation Accuracy")
plt.ylabel("Validation accuracy")
plt.xlabel("Epochs")
plt.legend(["acc","val_acc"])
plt.show()

The model is training well. The validation loss is continuously falling and has the same shape as training loss, signifying that no overtraining takes place. The validation loss is below the training loss becuase we include regularisation through dropout while training. 

The accuracy curves show consistent improvement. The accuracy acheived is reasonably high given that we are classifying between 4 different particles where two pairs of particles have very similar features considering only the enegy deposits alone. However, the model still obtains a meaningful discriminating power through the training. It does considerably better than guessing which would give only 25% accuracy.

The validation accuracy is above the training accuracy due to including regularisation during training in the form of dropout. 

The best val_accuracy achieved by the model is 0.5818

## Step 3 - Data validation (20%)

### Testing the performance of the network

In [ ]:
# Determine the likelihood for each particle initiating each test event
score = model.predict(x_test)

In [ ]:
# Find the particle ID predicted with highest likelihood
prediction = np.argmax(model.predict(x_test),axis=1)

In [ ]:
cm = confusion_matrix(y_test,prediction,normalize="true")

disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=["Electrons","Photons","Protons","Neutrons"])

disp.plot()
plt.title("Confusion matrix")
plt.show()

From the is confusion matrix we can see quite an interesting result.

The network often finds that classifying all of the electrons and photons as photons achieves the highest accuracy.

Training the model different times usually produces an alternation between predicting all electrons and photons and photons and predicting all electrons and photons as electrons. This shows that the information taken from the collisions is very similar between photons and electrons and the classifier cannot distinguish between the two. 

This is largely because the electrons and photons are both completely stopped by the ECAl in almost all cases. Both particle have the same energy and therefore nothing else can distinguish between them.


In the case of the protons and neutrons, the classifier gets a very high accuracy for the protons but much lower for the neutrons. In the course of this it decides to classify many of the neutrons as protons. It looks like the classifier is more confindent in classfying protons so it prioritises classying proton-like particles as protons to achieve a higher accuracy.

Through different trainings it has been found that the classification between protons and neutrons can alternate in a similar way to how it alternates for electrons and photons.

Here we can see that the classifier has the most difficulty classifying between electrons/photons and protons/neutrons. This is largely because without knowing their charges these sets of particles behave in a very similar way.

In [ ]:
bins = np.linspace(200,5190,20)

In [ ]:
correct = np.histogram(test_energy,bins= np.linspace(200,5190,20),weights=((y_test==prediction)*1).values)[0]
total = np.histogram(test_energy,bins= np.linspace(200,5190,20))[0]
bin_width = bins[1]-bins[0]
bin_centres = np.linspace(bins[0]+(bin_width/2),bins[-1]-(bin_width/2),19)
plt.hist(bin_centres,bins=bins,weights=correct/total)
plt.title("Accuracy as a function of energy")
plt.xlabel("Energy [MeV]")
plt.ylabel("Accuracy")
plt.show()

From this plot we can see that the accuracy has a weak relationship with the energy for the data we provide.

However there is a slight trend of increase in performance with energy, up to a certain point. 

This is because at low energies electrons, photons and most protons are stopped by the EM caloriemeter, however, at higher energies only the electrons and photons are stopped and protons and neutrons continue onwards. Therefore at higher energies the particles begin to show more different behaviour compared to each other.

### Resolution of energy measurements

Calculating the calibration quality for each event

In [ ]:
def resolution(ID):
    E_true = particles[particles.ID==ID][col_names[0]]
    E_detected = particles[particles.ID==ID][layers].values.sum(axis=1)
    
    # Dealing with missing energy values
    ratio = E_true/E_detected
    ratio=ratio.values
    ratio[np.where(ratio==np.inf)[0]]=0
    
    
    calibration = np.average(ratio)
    E_calibrated = E_detected*calibration
    calibration_quality = (E_calibrated-E_true)/E_true
    return E_true, calibration, calibration_quality

In [ ]:
#Plotting the 2D Histograms
fig,ax = plt.subplots(2,2,figsize=(12,9))

e_E_true, e_calibration, e_calibration_quality = resolution(0) 
g_E_true, g_calibration, g_calibration_quality = resolution(1) 
p_E_true, p_calibration, p_calibration_quality = resolution(2) 
n_E_true, n_calibration, n_calibration_quality = resolution(3) 

ax[0,0].set_title("Calibration quality of electrons")
ax[0,0].hist2d( x=e_E_true, y=e_calibration_quality, bins=(10, 20) )
ax[0,0].set_ylabel("($E_{cal}-E_{true})/E_{true}$")
ax[0,0].set_xlabel("True energy [MeV]")

ax[0,1].set_title("Calibration quality of photons")
ax[0,1].hist2d( x=g_E_true, y=g_calibration_quality, bins=(10, 20) )
ax[0,1].set_ylabel("($E_{cal}-E_{true})/E_{true}$")
ax[0,1].set_xlabel("True energy [MeV]")

ax[1,0].set_title("Calibration quality of protons")
ax[1,0].hist2d( x=p_E_true, y=p_calibration_quality, bins=(10, 20) )
ax[1,0].set_ylabel("($E_{cal}-E_{true})/E_{true}$")
ax[1,0].set_xlabel("True energy [MeV]")

ax[1,1].set_title("Calibration quality of neutrons")
ax[1,1].hist2d( x=n_E_true, y=n_calibration_quality, bins=(10, 20) )
ax[1,1].set_ylabel("($E_{cal}-E_{true})/E_{true}$")
ax[1,1].set_xlabel("True energy [MeV]")

plt.tight_layout()
plt.show()


Caribration quality is very good for electrons and photons as they get stopped completely in the ECAL. These particles also have a very good energy resolution. This can be seen through the sharp line in their respective plots.

The calibration quality for protons is worse likely because they deposit a lower fraction of their energy in the detectors. The resolution is also lower as can be seen through the more smeared out line.

The neutrons have the worst calibration quality as they interact the least with the detectors and deposit the least energy. This can be seen through less consistent calibration quality and hence most smeared out line, producing the lowest resolution.

In some cases the neutrons leave no energy deposit in the detector so these cases need to be treated with care during calibration. This is also the likeliest reason the mean of the calibration quality is significantly shifted from zero.

## Step 4 - Improvement and discussion (40%)

Improvemnts:

Add silicon tracking layer (1cm) to determine if the particle entering ECAL is charged or not (better discrimination between electrons/photons and protons/neutrons)

Increase lead to lAr ratio (5cm,25cm) in HCAL and increase layers to 8 (should allow larger fraction of energy to be deposited by protons and neutrons to increase energy resolution)

Add a magnetic field of 0.05 Tesla in x direction so the trajectory of charged particles will be curved. This should be able to provide more information to the machine learning model from the position of the hits in the silicon detector, and whether there have been any hits at all.

In [ ]:
new_col_names = ["TrueEnergy", "ECAL","HLayer1", "HLayer2", "HLayer3", "HLayer4", "HLayer5","HLayer6",
             "HLayer7","HLayer8"]

In [ ]:
new_layers = new_col_names[1:]

### Load the particles

In [ ]:
electrons_new = pd.read_csv("Electrons_improved.csv",comment="#",
                                header=None,names=new_col_names,dtype="float")

In [ ]:
photons_new = pd.read_csv("Photons_improved.csv",comment="#",
                                header=None,names=new_col_names,dtype="float")

In [ ]:
protons_new = pd.read_csv("Protons_improved.csv",comment="#",
                                header=None,names=new_col_names,dtype="float")

In [ ]:
neutrons_new = pd.read_csv("Neutrons_improved.csv",comment="#",
                                header=None,names=new_col_names,dtype="float")

### Load the tracking information

In [ ]:
tracker_cols = ["ID","Theta","Phi"]

In [ ]:
electrons_tracks = pd.read_csv("Electrons_tracker.csv",comment="#",
                                header=None,names=tracker_cols,dtype="float")

In [ ]:
photons_tracks = pd.read_csv("Photons_tracker.csv",comment="#",
                                header=None,names=tracker_cols,dtype="float")

In [ ]:
protons_tracks = pd.read_csv("Protons_tracker.csv",comment="#",
                                header=None,names=tracker_cols,dtype="float")

In [ ]:
neutrons_tracks = pd.read_csv("Neutrons_tracker.csv",comment="#",
                                header=None,names=tracker_cols,dtype="float")

In [ ]:
Event_IDs = np.int64(electrons_tracks.ID.unique())

In [ ]:
# Introducing new variables based on the tracking detector.
# The new variables should allow for an improved distinction between differently charged particles 
# and charged particles of different energies and masses. 

def pre_process(tracks, energies, Event_IDs):
    event_hits = pd.DataFrame(columns=["Event_ID","N_hits","Theta1","Phi1"])
    for i in Event_IDs:
        first_hit_theta =  tracks[tracks.ID==i].Theta.values[0]
        first_hit_phi = tracks[tracks.ID==i].Phi.values[0]
        n_hits = tracks[tracks.ID==i].Theta.values.size

        # Saving hit data to dataframe
        event_hits.loc[len(event_hits)]=[i,n_hits,first_hit_theta,first_hit_phi]
    tracks_energy = pd.concat([event_hits,energies],axis=1)
    tracks_energy.drop("Event_ID",axis=1,inplace=True)
    True_energy = tracks_energy.pop("TrueEnergy")
    tracks_energy.insert(0, True_energy.name, True_energy)
    
    return tracks_energy

In [ ]:
e_track_energy = pre_process(electrons_tracks,electrons_new,Event_IDs)
g_track_energy = pre_process(photons_tracks,photons_new,Event_IDs)
p_track_energy = pre_process(protons_tracks,protons_new,Event_IDs)
n_track_energy = pre_process(neutrons_tracks,neutrons_new,Event_IDs)

### Give labels

Particle IDs

electron = 0, photon = 1, proton = 2, neutron = 3

In [ ]:
e_track_energy.insert(loc=0,column="ID",value=np.zeros(e_track_energy.shape[0]))
g_track_energy.insert(loc=0,column="ID",value=np.ones(g_track_energy.shape[0]))
p_track_energy.insert(loc=0,column="ID",value=2*np.ones(p_track_energy.shape[0]))
n_track_energy.insert(loc=0,column="ID",value=3*np.ones(n_track_energy.shape[0]))

### Combine datasets

In [ ]:
particles_new = pd.concat([e_track_energy,g_track_energy,p_track_energy,n_track_energy],ignore_index=True)
particles_new.reset_index(drop=True, inplace=True)

In [ ]:
particles_new

### Create training and testing datasets

In [ ]:
# Split into features and target sets
features = particles_new[particles_new.columns[2:]]
target = particles_new.ID

In [ ]:
# Standarise data and split into training and test set
sc = preprocessing.StandardScaler()
features = sc.fit_transform(features)
# set random seed
seed = 1
# train - test split of dataset
x_train, x_test, y_train, y_test, train_energy, test_energy = \
model_selection.train_test_split(features, target, particles.TrueEnergy, test_size=0.3,random_state = seed)
print (x_train.shape, x_test.shape, y_train.shape,y_test.shape)

In [ ]:
def my_model(num_inputs, num_nodes, extra_depth,drop_out):
    # create model
    model = Sequential()
    # Input shape equal to number of features (6 detector readouts)
    model.add(Dense(num_nodes, input_dim=num_inputs, kernel_initializer="normal", activation="relu"))
    # Add dropout to prevent overtraining
    model.add(Dropout(drop_out))
    for i in range (extra_depth):
        # Adding dense layers
        model.add(Dense(num_nodes, kernel_initializer="normal", activation="relu"))
        # Following each dense layer by dropout layer
        model.add(Dropout(drop_out))
    model.add(Dense(4, activation="softmax",kernel_initializer="normal"))
    # Compile model to use multi-class loss
    model.compile(loss="sparse_categorical_crossentropy",optimizer="adam", metrics =["accuracy"])
    return model

In [ ]:
# Create the model with chosen hyper parameters
num_nodes = 50
extra_depth = 2
drop_out=0.2
model = my_model(x_train.shape[1],num_nodes,extra_depth,drop_out)

In [ ]:
model.summary()

In [ ]:
# Set training hyperparameters
batchSize = 50
N_epochs = 50

In [ ]:
# train the model and save the training history
history = model.fit(x_train,y_train,batch_size=batchSize,epochs=N_epochs,verbose=1,
                    validation_data=(x_test,y_test))

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend(["loss","val_loss"])
plt.show()


plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model Validation Accuracy")
plt.ylabel("Validation accuracy")
plt.xlabel("Epochs")
plt.legend(["acc","val_acc"])
plt.show()

The model now does much better on the new training data. All of the particle have distinct properties which can differentiate them now. Therefore the model achieves very high classification accuracy, where it can predit almost any event with complete certainty. After including the tracking information the problem to classfy the particles becomes much easier. The best val_accuracy achieved by the model is 0.9984.

However, in this case the model can become too complex for the easy problem at hand and can start to overtrain if left to train for too long. This is not a problem as training can be stopped earlier and best model performance can be saved before overtraining begins. 

Once again the validation loss and validation accuracy are above the loss and accuracy as regularisation in the form of drop-out is used.

## Step 3 - Data validation (20%)

### Testing the performance of the network

In [ ]:
# Determine the likelihood for each particle initiating each test event
score = model.predict(x_test)

In [ ]:
# Find the particle ID predicted with highest likelihood
prediction = np.argmax(model.predict(x_test),axis=1)

In [ ]:
cm = confusion_matrix(y_test,prediction,normalize="true")

disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=["Electrons","Photons","Protons","Neutrons"])

disp.plot()
plt.title("Confusion matrix")
plt.show()

This confusion matrix shows a much more different results compared to the previous one.

We can see that across all classes there is effectively 100% classification accuracy.

In a very few cases neutrons have been interpreted as photons and in even fewer cases photons have been interpreted as neutrons. This shows that there could be some possible room to improve distinction between these two particles by adding more detector components.

Apart from this the current detector layout works very well for this set of particle events.

In [ ]:
bins = np.linspace(200,5190,20)

In [ ]:
correct = np.histogram(test_energy,bins= np.linspace(200,5190,20),weights=((y_test==prediction)*1).values)[0]
total = np.histogram(test_energy,bins= np.linspace(200,5190,20))[0]
bin_width = bins[1]-bins[0]
bin_centres = np.linspace(bins[0]+(bin_width/2),bins[-1]-(bin_width/2),19)
plt.hist(bin_centres,bins=bins,weights=correct/total)
plt.title("Accuracy as a function of energy")
plt.xlabel("Energy [MeV]")
plt.ylabel("Accuracy")
# plt.xscale("log")
plt.show()

From this plot we can see a similar trend to the previous detector layout. 

The accuracy is visibly decreased at the lowest energies and at higher energy we achieve close to 100% accuracy. 

This is because at low energies electrons, photons and most protons are stopped by the EM caloriemeter, however, at higher energies only the electrons and photons are stopped and protons and neutrons continue onwards. Therefore at higher energies the particles begin to show more different behaviour compared to each other.

However, there is an interesting decrease in accuracy near 3.5 GeV. This is likely near the point where neutrons experience lowest energy loss through ionisation, but this is only my guess.

### Resolution of energy measurements

Calculating the calibration quality for each event

In [ ]:
def resolution(ID):
    E_true = particles_new[particles_new.ID==ID][new_col_names[0]]
    E_detected = particles_new[particles_new.ID==ID][new_layers].values.sum(axis=1)
    
    # Dealing with missing energy values
    ratio = E_true/E_detected
    ratio=ratio.values
    ratio[np.where(ratio==np.inf)[0]]=0
    
    
    calibration = np.average(ratio)
    E_calibrated = E_detected*calibration
    calibration_quality = (E_calibrated-E_true)/E_true
    return E_true, calibration, calibration_quality

In [ ]:
#Plotting the 2D Histograms
fig,ax = plt.subplots(2,2,figsize=(12,9))

e_E_true, e_calibration, e_calibration_quality = resolution(0) 
g_E_true, g_calibration, g_calibration_quality = resolution(1) 
p_E_true, p_calibration, p_calibration_quality = resolution(2) 
n_E_true, n_calibration, n_calibration_quality = resolution(3) 

ax[0,0].set_title("Calibration quality of electrons")
ax[0,0].hist2d( x=e_E_true, y=e_calibration_quality, bins=(10, 20) )
ax[0,0].set_ylabel("($E_{cal}-E_{true})/E_{true}$")
ax[0,0].set_xlabel("True energy [MeV]")

ax[0,1].set_title("Calibration quality of photons")
ax[0,1].hist2d( x=g_E_true, y=g_calibration_quality, bins=(10, 20) )
ax[0,1].set_ylabel("($E_{cal}-E_{true})/E_{true}$")
ax[0,1].set_xlabel("True energy [MeV]")

ax[1,0].set_title("Calibration quality of protons")
ax[1,0].hist2d( x=p_E_true, y=p_calibration_quality, bins=(10, 20) )
ax[1,0].set_ylabel("($E_{cal}-E_{true})/E_{true}$")
ax[1,0].set_xlabel("True energy [MeV]")

ax[1,1].set_title("Calibration quality of neutrons")
ax[1,1].hist2d( x=n_E_true, y=n_calibration_quality, bins=(10, 20) )
ax[1,1].set_ylabel("($E_{cal}-E_{true})/E_{true}$")
ax[1,1].set_xlabel("True energy [MeV]")

plt.tight_layout()
plt.show()


Caribration quality is very good for electrons and photons. However the calibration quality of the electrons now varies more with energy. This is most likely because the electrons interact differently with the silicon detector at different energies. However the calibration quality is still very consistent at each energy (high resolution) for both electrons and photons which is seen through the sharp lines.

We can also see that the calibration quality for protons and neutrons has significantly improved. The calibration quality varies a lot less with energy compared to the previous detector layout. This is largely due to the increase in absorber thickness and number of HCAL layers. Furthermore, we can see that the energy resolution has also improved due to the sharper lines for both plots. 

We can however still see that the mean calibration quality for neutrons is shifted from zero. This indicates that a fraction neutrons still pass through the detector without depositing energy.